In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam

In [ ]:
# Förbered area_df
area_df = pd.read_csv(r'P:\Workspace\Forbrukningsestimering\kunder.csv')
area_df.rename(columns={'GATUADRESS' : 'Adress',
'FASTIGHET' : 'Fastighetsbeteckning','TOTALAREAL' : 'Area'}, inplace=True)

area_df['Adress'] = area_df['Adress'].str.lower()
area_df['Adress'] = area_df['Adress'].str.rstrip()
area_df['Adress'] = area_df['Adress'].str.lstrip()

area_df['Fastighetsbeteckning'] = area_df['Fastighetsbeteckning'].str.lower()
area_df['Fastighetsbeteckning'] = area_df['Fastighetsbeteckning'].str.rstrip()
area_df['Fastighetsbeteckning'] = area_df['Fastighetsbeteckning'].str.lstrip()

area_df['Area'].fillna(0.0, inplace=True)

In [ ]:
# Förbered merged_df
merged_df = pd.read_csv(r'P:\Workspace\update_merge\merged.csv')
merged_df.rename(columns={'contextMap.street' : 'Adress', 'name' : 'sensorname',
'contextMap.Fastighetsbeteckning' : 'Fastighetsbeteckning',
'contextMap.Mätarstorlek' : 'Mätarstorlek',
'contextMap.Fastighetstyp' : 'Fastighetstyp',
'contextMap.city' : 'Postort'}, inplace=True)

merged_df['Adress'] = merged_df['Adress'].str.lower()
merged_df['Adress'] = merged_df['Adress'].str.rstrip()
merged_df['Adress'] = merged_df['Adress'].str.lstrip()

merged_df['Fastighetsbeteckning'] = merged_df['Fastighetsbeteckning'].str.lower()
merged_df['Fastighetsbeteckning'] = merged_df['Fastighetsbeteckning'].str.rstrip()
merged_df['Fastighetsbeteckning'] = merged_df['Fastighetsbeteckning'].str.lstrip()

merged_df = merged_df.loc[merged_df['Fastighetstyp'] == 'VILLA']

In [ ]:
# Förbered EDP
edp_df = pd.read_csv(r'P:\Workspace\update_merge\EDP.csv', dtype=str)
edp_df['Mätarnr'] = edp_df['Mätarnr'].str[1:]
edp_df['Mätarnr'] = 'VM_' + edp_df['Mätarnr']
edp_df.rename(columns= {'Mätarnr' : 'sensorname'}, inplace=True)

In [ ]:
# Förbered bebyggelsetyp
bebyggelse_df = pd.read_csv(r'P:\Workspace\forbrukningsestimering\Bebyggelsetyp.csv', sep=';', dtype=str)
bebyggelse_df = bebyggelse_df[['FASTIGHET', 'sub_c_type']]
bebyggelse_df.rename(columns={'FASTIGHET' : 'Fastighetsbeteckning', 'sub_c_type' : 'Bebyggelsetyp'}, inplace=True)

bebyggelse_df['Fastighetsbeteckning'] = bebyggelse_df['Fastighetsbeteckning'].str.lower()
bebyggelse_df['Fastighetsbeteckning'] = bebyggelse_df['Fastighetsbeteckning'].str.rstrip()
bebyggelse_df['Fastighetsbeteckning'] = bebyggelse_df['Fastighetsbeteckning'].str.lstrip()

bebyggelse_df['Bebyggelsetyp'] = bebyggelse_df['Bebyggelsetyp'].str.replace(' - Grafik', '')

In [ ]:
# Förbered tryck
tryck_xls = pd.ExcelFile(r'P:\Workspace\forbrukningsestimering\Adresslista med tryck.xlsx', engine='openpyxl')
malmö_tryck_df = pd.read_excel(tryck_xls, sheet_name='Malmö', usecols=[5,6,13], names=['Fastighetsbeteckning','Adress', 'Tryck'])
lund_tryck_df = pd.read_excel(tryck_xls, sheet_name='Lund', usecols=[6,7,13], names=['Fastighetsbeteckning','Adress', 'Tryck'])
lomma_tryck_df = pd.read_excel(tryck_xls, sheet_name='Lomma', usecols=[6,7,13], names=['Fastighetsbeteckning','Adress', 'Tryck'])
eslöv_tryck_df = pd.read_excel(tryck_xls, sheet_name='Eslöv', usecols=[6,7,13], names=['Fastighetsbeteckning','Adress', 'Tryck'])
burlöv_tryck_df = pd.read_excel(tryck_xls, sheet_name='Burlöv', usecols=[7,8,13], names=['Fastighetsbeteckning','Adress', 'Tryck'])

tryck_df = pd.concat([malmö_tryck_df, lund_tryck_df, lomma_tryck_df, eslöv_tryck_df, burlöv_tryck_df])

tryck_df['Adress'] = tryck_df['Adress'].str.lower()
tryck_df['Adress'] = tryck_df['Adress'].str.rstrip()
tryck_df['Adress'] = tryck_df['Adress'].str.lstrip()

tryck_df['Fastighetsbeteckning'] = tryck_df['Fastighetsbeteckning'].str.lower()
tryck_df['Fastighetsbeteckning'] = tryck_df['Fastighetsbeteckning'].str.rstrip()
tryck_df['Fastighetsbeteckning'] = tryck_df['Fastighetsbeteckning'].str.lstrip()

tryck_df['Tryck'] = tryck_df['Tryck'].replace(',','.',regex=True).astype(float)

In [ ]:
# Förbered data_df och label_df
data_df = pd.read_csv(r'P:\Workspace\influx_data\water_use.csv')
data_df = data_df[['_time', '_value', 'sensorname']]
data_df.dropna(inplace=True)

data_df['_time'] = pd.to_datetime(data_df['_time'], format='%Y-%m-%d %H:%M:%S')
data_df['_value'] = data_df.groupby(['sensorname'])['_value'].diff()
data_df.dropna(inplace=True)
data_df['date'] = data_df['_time'].dt.date
data_df['_value'] = data_df.groupby(['sensorname','date'])['_value'].transform('sum')
data_df['weekday'] = data_df['_time'].dt.day_name()
data_df.drop_duplicates(subset=['sensorname', 'date'], inplace=True)
data_df.drop(columns=['_time', 'date'], inplace=True)
data_df['_value'] = data_df.groupby(['sensorname', 'weekday'])['_value'].transform('mean')

data_df.drop_duplicates(inplace=True)
data_df.reset_index(inplace=True)
data_df.drop(columns=['index'], inplace=True)

data_df = pd.merge(data_df, merged_df[['sensorname', 'Fastighetsbeteckning', 'Adress', 'Mätarstorlek', 'Postort']], on='sensorname', how='inner')
data_df = pd.merge(data_df, area_df[['Fastighetsbeteckning', 'Adress', 'Area']], on=['Fastighetsbeteckning', 'Adress'], how='inner')
data_df = pd.merge(data_df, bebyggelse_df, on='Fastighetsbeteckning', how='inner')
data_df = pd.merge(data_df, edp_df[['sensorname', 'SenÅrsförbr']], on='sensorname', how='inner')
data_df = pd.merge(data_df, tryck_df, on=['Fastighetsbeteckning', 'Adress'], how='inner')

corrupt_list= ['VM_05351860', 'VM_05353028', 'VM_05353131', 'VM_05353350', 'VM_05353365', 'VM_05354170', 'VM_05354445', 'VM_05450393', 'VM_05450593', 'VM_05545983', 'VM_05545950', 'VM_05351958', 'VM_05450161', 'VM_05451196', 'VM_05688636']
data_df = data_df[~data_df['sensorname'].isin(corrupt_list)]

data_df.dropna(inplace=True)
data_df.reset_index(inplace=True)
data_df.drop(columns=['index'], inplace=True)

data_df = data_df.loc[data_df['_value'] > 0]
andel_att_ta_bort = int(0.05*len(data_df))

data_df.drop(data_df['_value'].nsmallest(andel_att_ta_bort).index, inplace=True)
data_df.drop(data_df['_value'].nlargest(andel_att_ta_bort).index, inplace=True)
data_df.reset_index(inplace=True, drop=True)

label_df = data_df[['Adress', 'Fastighetsbeteckning', 'weekday', '_value']]
label_df['_value'] = label_df.groupby(['Adress', 'Fastighetsbeteckning', 'weekday'])['_value'].transform('sum')
label_df.drop_duplicates(inplace=True)
label_df.reset_index(inplace=True, drop=True)

data_df.drop(columns=['_value'], inplace=True)
data_df['Antal DN20'] = data_df.loc[data_df['Mätarstorlek'] == 'DN20'].groupby(['Adress', 'Fastighetsbeteckning'])['sensorname'].transform('nunique')
data_df['Antal DN25'] = data_df.loc[data_df['Mätarstorlek'] == 'DN25'].groupby(['Adress', 'Fastighetsbeteckning'])['sensorname'].transform('nunique')
data_df['Antal DN20'].fillna(0, inplace=True)
data_df['Antal DN25'].fillna(0, inplace=True)

data_df['SenÅrsförbr'] = data_df['SenÅrsförbr'].astype(float)
data_df['SenÅrsförbr'] = data_df.groupby(['Adress', 'Fastighetsbeteckning'])['SenÅrsförbr'].transform('sum')
data_df['SenÅrsförbr'].fillna(0, inplace=True)

data_df.drop(columns=['sensorname','Mätarstorlek'], inplace=True)
data_df.drop_duplicates(subset=['Adress', 'Fastighetsbeteckning', 'weekday'], inplace=True)

data_df.sort_values(by=['Fastighetsbeteckning', 'Adress'], inplace=True)
data_df.reset_index(inplace=True, drop=True)

#Fastighetstyp
#unique_fast = data_df['Fastighetstyp'].unique().tolist()
#for fast in unique_fast:
#    data_df[fast] = 0
#    data_df.loc[data_df['Fastighetstyp'] == fast, fast] = 1
#
#data_df.drop(columns=['Fastighetstyp'], inplace=True)

#Postort
unique_post = data_df['Postort'].unique().tolist()
for ort in unique_post:
    data_df[ort] = 0
    data_df.loc[data_df['Postort'] == ort, ort] = 1

data_df.drop(columns=['Postort'], inplace=True)

#Bebyggelsetyp
unique_bebyggelse = data_df['Bebyggelsetyp'].unique().tolist()
for typ in unique_bebyggelse:
    data_df[typ] = 0
    data_df.loc[data_df['Bebyggelsetyp'] == typ, typ] = 1

data_df.drop(columns=['Bebyggelsetyp'], inplace=True)

#Weekday
unique_weekday = data_df['weekday'].unique().tolist()
for day in unique_weekday:
    data_df[day] = 0
    data_df.loc[data_df['weekday'] == day, day] = 1
    
day_list = data_df['weekday'].tolist()

data_df.drop(columns=['weekday'], inplace=True)

In [ ]:
adress_list = data_df['Adress'].tolist()
fast_list = data_df['Fastighetsbeteckning'].tolist()

temp_customer_list = [list(x) for x in zip(adress_list, fast_list, day_list)]

label_list = []
customer_list = []
feature_list = []

for customer in temp_customer_list:
    customer_label = label_df.loc[(label_df['Adress'] == customer[0]) & (label_df['Fastighetsbeteckning'] == customer[1]) & (label_df['weekday'] == customer[2])]['_value'].tolist()
    label_list.append(customer_label)
    customer_list.append(customer)
    
    customer_feature = data_df.loc[(data_df['Adress'] == customer[0]) & (data_df['Fastighetsbeteckning'] == customer[1]) & (data_df[customer[2]] == 1)][[
        'Area', 'Antal DN20', 'Antal DN25', 'SenÅrsförbr', 
        'Bjärred', 'Dalby', 'Södra Sandby', 'Tygelsjö', 'Flyinge', 'Harlösa', 'Stehag', 'Klagshamn', 'Veberöd', 'Hurva', 
        'Torna-Hällestad', 'Åkarp', 'Lomma', 'Marieholm', 'Låg bebyggelse', 'Sluten bebyggelse', 'Industriområde', 
        'Hög bebyggelse', 'Annan öppen mark (yta)', ' ', 'Saturday', 'Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday',
        'Tryck']].values.tolist()[0]
    feature_list.append(customer_feature)
    
X = np.array(feature_list)
y = np.array(label_list)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [ ]:
np.mean(y)

In [ ]:
#Skapa modell
model = Sequential()
model.add(Dense(32, input_dim=32, kernel_initializer='normal', activation='relu'))
model.add(Dense(16, kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal', activation='softplus'))

opt = Adam(learning_rate=0.000002)
model.compile(loss='mse', optimizer=opt)

#Träna modellen
callback = EarlyStopping(monitor='loss', patience=20, restore_best_weights=True)
history = model.fit(X_train, y_train,validation_data=(X_test, y_test), epochs=1000, callbacks=[callback])
loss = history.history['loss']
val_loss = history.history['val_loss']

#Spara modellen
model.save(r'P:\Workspace\Forbrukningsestimering\model_forbrukning.h5', save_format='h5')

In [ ]:
#Plotta loss
epochs = range(1, len(loss)+1)
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.legend()
plt.show()

In [ ]:
#Plotta prediction exempel från testdatan
N_PLOTS=20

for i in range(N_PLOTS):
    prediction= model.predict(np.array([X_test[i],]))
    plt.figure
    plt.bar(0, y_test.tolist()[i], label='Real')
    plt.bar(1, prediction.tolist()[0], label='Prediction')
    plt.legend()
    print('****************')
    print('Real:', y_test.tolist()[i][0], 'm^3','| Prediction:', prediction.tolist()[0][0],'m^3')
    print('Skillnad:', 1000*(max([y_test.tolist()[i][0], prediction.tolist()[0][0]]) - min([y_test.tolist()[i][0], prediction.tolist()[0][0]])), 'liter')
    plt.show()

In [ ]:
#Beräkna det totala felet för alla fastigheter i test

sum_skillnad = 0
avg_skillnad = 0

print('Antal mätare:', len(X_test))
for i in range(len(X_test)):
    prediction= model.predict(np.array([X_test[i],]))
    sum_skillnad += prediction.tolist()[0][0] - y_test.tolist()[i][0]
    avg_skillnad += max([y_test.tolist()[i][0], prediction.tolist()[0][0]]) - min([y_test.tolist()[i][0], prediction.tolist()[0][0]])
    
avg_skillnad /= len(X_test)

print('Totalt fel:', sum_skillnad, 'm^3')
print('Medelfel per fastighet och dag:', avg_skillnad, 'm^3')